In [1]:
import Utils.FS.file as file
import TextPreprocess.cleaner as cleaner
from sklearn.feature_extraction.text import CountVectorizer
import Utils.misc as helper
import Utils.pandas_helper as ph

In [2]:
eng_link_posts = file.read('eng_link_posts_2.pkl')

In [3]:
print(eng_link_posts.shape)
eng_link_posts[['id', 'name', 'message', 'description']].sample(10)

(7695711, 23)


,id,name,message,description
1884124,317121371634190_1453303041349345,giphy.com,#MakeLifeFunnier is giving u a chance to win B...,NaN
4148853,112681712076876_1417417975163072,A que no te atreves a verlo ??,YO no pude :( y tu ?,VER YA !!
4078429,56571674793_10154461347059794,Hands On Product Reviews December 2016,This LED light is small enough for your first-...,Check out the latest products reviewed in JEMS.
4237772,295830058648_10154555269078649,Living with HIV,Living with HIV http://bbc.in/2kMqm7z,An Australian artist aims to rewrite the conve...
4838325,59651918044_10152988588953045,"Former All-Stars Candace Parker, Sylvia Fowles...","Not the result we wanted on Wednesday, but fel...",After sitting out the first half of the 2015 W...
1098593,12786606933_226909467354497,"Noah and the Whale, 'Life Is Life (Yuksek Remi...",The Yuksek remix of ‘Life is Life’ is the MP3 ...,Listen to Noah and the Whale's 'Life is Life' ...
7617924,146205388831743_799035300215412,[VIDEO] Mickey ‘Memphitz’ Wright Wanted For Ag...,NaN,Surveillance video shows the moments police sa...
7463634,141224181394_10153633728576395,HIJAB - Great Britain | Islamic Design House,Do you know our InEssence plain scarves? They ...,NaN
7674407,139399349522670_773853032743962,"Deadpool: Ryan Reynolds on that R-rating, the ...",Glad it's not PG-13...ugh could you imagine th...,"""Deadpool's greatest superpower is annoying th..."
475008,371844480840_10154266786875841,[VIDEO] Safaree Samuels Says Nicki Minaj Cheat...,Safaree Ain't Lettin No One Stop His Hustle! 😂,Get Your Tea Cups Ready!


In [ ]:
eng_link_posts['clean_description'] = ph.parallel_apply(eng_link_posts['description'].astype(str), cleaner.clean)

In [ ]:
eng_link_posts[['clean_description', 'description']].sample(10)

In [ ]:
def clean(df):
    df['clean_description'] = df['description'].astype(str) \
        .apply(helper.trace_progress(cleaner.clean))
    return df

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

print ('parallel version: ')
# 4 min 30 seconds
%time result = applyParallel(eng_link_posts.groupby(eng_link_posts.index % multiprocessing.cpu_count()), clean)

In [ ]:
result[['description', 'clean_description']].sample(100)

In [ ]:
import pandas as pd
posts = pd.DataFrame()
for i in range(92):
    print('./posts-{:0>12d}.gzip.csv'.format(i))
    p = pd.read_csv('./posts-{:0>12d}.gzip.csv'.format(i))
    p = p[p['type'] == 'link']
    posts = posts.append(p, ignore_index=True)
    print(posts.shape)

In [ ]:
posts

In [ ]:
import pickle
with open('eng_link_posts_2.pkl', 'wb') as f:
    pickle.dump(posts, f)

In [ ]:
import Utils.GoogleCloud.storage as gcs
gcs.upload('facebook-posts-processed', 'eng_link_posts_2.pkl', 'eng_link_posts_2.pkl')

In [ ]:
lh_posts = posts[posts['page_id'].astype(str) == '109565802405671']

In [ ]:
lh_posts[['id', 'name', 'description', 'caption']]

In [ ]:
lh_posts.shape